In [1]:
%pip install mediapipe==0.10.11

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
import random
import gc
import glob
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import Image, clear_output
from torch.cuda import memory_allocated, empty_cache
from torch.optim import Adam
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

%matplotlib inline

In [4]:
data_root = r'E:\project\New_Data'
file_root = rf'{data_root}\Image'
project_name = 'fall_detection'
print(file_root)
train_root = rf'{data_root}\train'
valid_root = rf'{data_root}\valid'
test_root = rf'{data_root}\test'
cls_list = ['Normal', 'Fall']
cls_filename_list = ['N', 'BY', 'FY', 'SY']

image_root = f'{train_root}\\images'
label_root = f'{train_root}\\labels'

image_list = sorted(glob.glob(f'{image_root}\\*.jpg'))
print(len(image_list))
label_list = [x.replace('images', 'labels').replace('JPG', 'jpg').replace('jpg', 'txt') for x in image_list]
print(len(image_list), len(label_list))

E:\project\New_Data\Image
67840
67840 67840


In [5]:
def image_sampling(image_root, label_root, sample_num):
    image_cls_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_names_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    image_sampled_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    label_sampled_dict = {'N': [], 'BY': [], 'FY': [], 'SY': []}
    for cls in cls_filename_list:
        image_cls_dict[cls] = glob.glob(f'{image_root}\\*_{cls}_*')
        image_names_dict[cls] = list(set([x.split('\\')[-1].split('_')[0] for x in image_cls_dict[cls]]))
        if cls == 'N':
            div = 2
        else:
            div = 6
        image_names_dict[cls] = random.sample(image_names_dict[cls], (sample_num // 80) // div)
        for i in tqdm(image_names_dict[cls]):
            image_sampled_dict[cls].extend(sorted(glob.glob(f'{image_root}\\{i}_*')))
            label_sampled_dict[cls].extend(sorted(glob.glob(f'{label_root}\\{i}_*')))
    print([image_sampled_dict[cls][:10] for cls in cls_filename_list])
    print([label_sampled_dict[cls][:10] for cls in cls_filename_list])
    print([len(image_sampled_dict[cls]) for cls in cls_filename_list])
    return image_sampled_dict, label_sampled_dict

In [6]:
image_dict, label_dict = image_sampling(image_root, label_root, 12000)
len(label_dict['N'])

100%|██████████| 25/25 [00:03<00:00,  6.83it/s]

[['E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I001.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I002.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I003.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I004.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I005.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I006.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I007.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I008.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I009.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I010.jpg'], ['E:\\project\\New_Data\\train\\images\\01788_Y_E_BY_C1_I001.jpg', 'E:\\project\\New_Data\\train\\images\\01788_Y_E_BY_C1_I002.jpg', 'E:\\project\\New_Data\\train\\images\\01788_Y_E_BY_C1_I003.jpg', 'E:\\project\\New_Data\\train\\images\\01788_Y_E_BY_C1_I004.jpg', 'E:\\project\\New_Data\\train\\images\\01788_Y_E_BY_C1_I005.jpg', 'E:\\project\\Ne

6000

In [7]:
BATCH_SIZE = 8
EPOCH = 2
NUM_LAYERS = 1
n_CONFIDENCE = 0.3


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False, min_detection_confidence=n_CONFIDENCE)
attention_dot = [n for n in range(29)]


draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], [12, 14], [14, 16],
             [16, 22], [16, 20], [16, 18], [18, 20], [23, 25], [25, 27], [24, 26], [26, 28],
             [11, 12], [11, 23], [23, 24], [12, 24], [9, 10], [0, 5], [0, 2], [5, 8], [2, 7]]

In [8]:
def get_pose(image_list: list, label_list: list, label_type: str, attention_dot: list, draw_line: list, isSample=False, sample_size=0, pose=pose):
    if isSample:
        print('sample mod')
        print(len(image_list))
        plt.figure(figsize=(8, 4*sample_size))
    frame_length = 10 # LSTM 모델에 넣을 frame 수
    xy_list_list = []
    omission_cnt = 0
    # pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False, min_detection_confidence=n_CONFIDENCE)
    for i, (img_src, lbl) in enumerate((zip(image_list, label_list))):
        if label_type == 'yolo':
            with open(lbl, 'r', encoding='utf-8') as f:
                label_str = f.read()
                bbox = list(map(float, label_str.split(' ')[1:]))

            bbox = [int(x * 640) for x in bbox]
            bbox = [
                max(int(bbox[0] - bbox[2] / 2 - 30), 0),
                max(int(bbox[1] - bbox[3] / 2 - 30), 0),
                min(int(bbox[0] + bbox[2] / 2 + 30), 640),
                min(int(bbox[1] + bbox[3] / 2 + 30), 640),
            ]
            img = cv2.imread(img_src)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if isSample:
                bbox_img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 0), 2)
                cv2.putText(bbox_img, img_src.split('\\')[-1], (5, 30), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), lineType=cv2.LINE_AA)
                plt.subplot(sample_size, 2, (2*i+1))
                plt.imshow(bbox_img)

        
        """ Yolov5 바운딩 박스 좌표 안에서 mediapipe Pose 추출"""

        # img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2) # 바운딩 박스 그리기 : 데이터 추출 확인용
        c_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]] # 바운딩 박스 좌표
        results = pose.process(c_img) # Yolov5 바운딩 박스 좌표 안에서 'mp_pose' 좌표

        if not results.pose_landmarks: continue
        idx = 0
        draw_line_dic = {}
        xy_list = []
        # 33 반복문 진행 : 33개 중 18개의 dot
        for x_and_y in results.pose_landmarks.landmark:
            if idx in attention_dot:
                xy_list.append(x_and_y.x)
                xy_list.append(x_and_y.y)

                x, y = int(x_and_y.x * (bbox[2] - bbox[0])), int(x_and_y.y * (bbox[3] - bbox[1]))
                draw_line_dic[idx] = [x, y]
            idx += 1

        if len(xy_list) != len(attention_dot) * 2:
            print('Error : attention_dot 데이터 오류')

        xy_list_list.append(xy_list)

        """mediapipe line 그리기 부분 : 데이터 추출(dot) 확인용"""
        if isSample:
            for line in draw_line:
                x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 4)
            plt.subplot(sample_size, 2, (2*i+2))
            plt.imshow(c_img)
        
    # 부족한 프레임 수 맞추기
    if isSample:
        if len(xy_list_list) < len(image_list) // 2:
            return False
        elif len(xy_list_list) < len(image_list):
            f_ln = len(image_list) - len(xy_list)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
    else:
        if len(xy_list_list) < 5:
            return False, 5 - len(xy_list_list)
        elif len(xy_list_list) < 10:
            f_ln = frame_length - len(xy_list)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
                omission_cnt = f_ln
    
    if isSample:
        plt.show()
    return xy_list_list, omission_cnt

In [9]:
def get_pose_sample(image_dict: dict, label_dict: dict, label_type: str, attention_dot: list, draw_line: list, sample_size: int, sample_start=0):
    image_pose_sample = []
    label_pose_sample = []
    for cls in cls_filename_list:
        div = 2 if cls == 'N' else 6
        image_pose_sample.extend(image_dict[cls][sample_start:(sample_start+sample_size//div)])
        label_pose_sample.extend(label_dict[cls][sample_start:(sample_start+sample_size//div)])
    print(len(image_pose_sample), 'asdf')
    pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False, min_detection_confidence=n_CONFIDENCE)
    xy_sample = get_pose(image_pose_sample, label_pose_sample, label_type, attention_dot, draw_line, True, sample_size, pose)

In [33]:
1/0
a = get_pose_sample(image_dict, label_dict, 'yolo', attention_dot, draw_line, sample_size=40)


ZeroDivisionError: division by zero

In [10]:
raw_data = []
image_pose_input = []
label_pose_input = []
for cls in cls_filename_list:
    img_name = {'index': 'None', 'direction': 'None'}
    img_bundle = []
    label_bundle = []
    isNewBundle = 0
    for img, lbl in zip(tqdm(image_dict[cls]), label_dict[cls]):
        # 처음 값 입력
        if img_name['index'] != img.split('\\')[-1].split('_')[0]:
            img_name['index'] = img.split('\\')[-1].split('_')[0]
            isNewBundle += 1
        if img_name['direction'] != img.split('\\')[-1].split('_')[-2]:
            img_name['direction'] = img.split('\\')[-1].split('_')[-2]
            isNewBundle += 1
        if isNewBundle > 0:
            if img_bundle != []:
                image_pose_input.append(img_bundle)
                label_pose_input.append(label_bundle)
            img_bundle = []
            label_bundle = []
            isNewBundle = 0
        img_bundle.append(img)
        label_bundle.append(lbl)


print(len(image_dict['N'])+len(image_dict['BY'])+len(image_dict['FY'])+len(image_dict['SY']))
print(image_pose_input[:3])
print(len(image_pose_input))
print(len(label_pose_input))

print(raw_data)

100%|██████████| 2000/2000 [00:00<00:00, 666556.06it/s]

12000
[['E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I001.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I002.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I003.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I004.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I005.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I006.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I007.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I008.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I009.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C1_I010.jpg'], ['E:\\project\\New_Data\\train\\images\\02502_H_A_N_C2_I001.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C2_I002.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C2_I003.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C2_I004.jpg', 'E:\\project\\New_Data\\train\\images\\02502_H_A_N_C2_I005.jpg', 'E:\\project\\N

In [11]:
del raw_data
gc.collect()
raw_data = []

In [12]:
a = [len(i) == 10 for i in image_pose_input]
print(a.count(True))

1588


In [16]:
# ommision_total = 0
# raw_data = list(map(
#     lambda img_list, lbl_list: {
#         'key': 0 if img_list[0].split('\\')[-1].split('_')[-3] == 'N' else 1,
#         'value': get_pose(img_list, lbl_list, 'yolo', attention_dot, draw_line, False)
#         },
#     image_pose_input, label_pose_input
#     ))

KeyboardInterrupt: 

In [28]:
def experiment():
    return [1,1,1], 0

In [12]:
ommision_total = 0
for img_list, lbl_list in zip(tqdm(image_pose_input), label_pose_input):
    cls = 0 if img_list[0].split('\\')[-1].split('_')[-3] == 'N' else 1
    pose_list, ommision_cnt = get_pose(img_list, lbl_list, 'yolo', attention_dot, draw_line, False, pose)
    if pose_list != False:
        raw_data.append({'key': cls, 'value': pose_list})
    del pose_list, ommision_cnt
    gc.collect()

print('ommision: ', ommision_total)


100%|██████████| 1196/1196 [10:07<00:00,  1.97it/s]

ommision:  0


In [16]:
print(raw_data[0])
random.shuffle(raw_data)
nd = 0
ad = 0
for i in range(len(raw_data)):
    if raw_data[i]['key'] == 0:
        nd += 1
    else:
        ad += 1
print('normal data:', nd, '| fall data:', ad)

{'key': 1, 'value': [[0.42174863815307617, 0.39682167768478394, 0.4331596791744232, 0.35357239842414856, 0.4459232687950134, 0.3624459505081177, 0.45696771144866943, 0.35902005434036255, 0.3984292149543762, 0.3622378408908844, 0.3938250243663788, 0.37350404262542725, 0.3832474648952484, 0.37446317076683044, 0.47753649950027466, 0.3479810655117035, 0.3714076280593872, 0.36939698457717896, 0.45053669810295105, 0.4097072184085846, 0.40958172082901, 0.4145335555076599, 0.5399222373962402, 0.39826542139053345, 0.3126985430717468, 0.4231944680213928, 0.5895161628723145, 0.5112392902374268, 0.313571959733963, 0.541357696056366, 0.5445574522018433, 0.5664742588996887, 0.3847618103027344, 0.6089386343955994, 0.5401108860969543, 0.5873429775238037, 0.39435330033302307, 0.6349895596504211, 0.5150372385978699, 0.5761933326721191, 0.41795235872268677, 0.6186273097991943, 0.5124875903129578, 0.5669752359390259, 0.4202782213687897, 0.6077748537063599, 0.5217708349227905, 0.5136910080909729, 0.4087259

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])

    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [19]:
split_ratio = [0.8, 0.1, 0.1]
train_len = int(len(raw_data) * split_ratio[0])
val_len = int(len(raw_data) * split_ratio[1])
test_len = len(raw_data) - train_len - val_len

print('{}, {}, {}'.format(train_len, val_len, test_len))

908, 113, 114


In [20]:
train_dataset = MyDataset(raw_data)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [21]:
# LSTM
class skeleton_LSTM(nn.Module):
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=len(attention_dot) * 2, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(32,2)

    def forward(self, x) :
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:,-1,:]) # x[배치 크기, 시퀀스 길이, 은닉 상태 크기], [:, -1, :] -> 마지막 시간 단계만 선택

        return x
     

In [22]:
class skeleton_GRU(nn.Module):
    def __init__(self):
        super(skeleton_GRU, self).__init__()
        self.GRU1 = nn.GRU(input_size=len(attention_dot) * 2, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.GRU2 = nn.GRU(input_size=128, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.GRU3 = nn.GRU(input_size=256, hidden_size=512, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.GRU4 = nn.GRU(input_size=512, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.GRU5 = nn.GRU(input_size=256, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.GRU6 = nn.GRU(input_size=128, hidden_size=64, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.GRU7 = nn.GRU(input_size=64, hidden_size=32, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(32,2)

    def forward(self, x) :
        x, _ = self.GRU1(x)
        x, _ = self.GRU2(x)
        x, _ = self.GRU3(x)
        x = self.dropout1(x)
        x, _ = self.GRU4(x)
        x, _ = self.GRU5(x)
        x, _ = self.GRU6(x)
        x = self.dropout2(x)
        x, _ = self.GRU7(x)
        x = self.fc(x[:,-1,:]) # x[배치 크기, 시퀀스 길이, 은닉 상태 크기], [:, -1, :] -> 마지막 시간 단계만 선택

        return x

In [23]:
# 모델 초기화

def init_model():
    global net, loss_fn, optim
    plt.rc('font', size = 10)
    net = skeleton_LSTM().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = Adam(net.parameters(), lr=0.0001)

# epoch 카운터 초기화
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

# 모든 Log를 초기화
def init_log():
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    plt.rc('font', size = 10)
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []

In [24]:
def record_train_log(_tloss, _tacc, _time):
    # Train Log 기록
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)

def record_valid_log(_vloss, _vacc):
    # Validation Log 기록
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)

def last(log_list):
    # last 안의 마지막 숫자를 반환(print_log 함수에서 사용)
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

def print_log():
    # 학습 추이 출력 : 소숫점 3자리까지
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)

    log_str = 'Epoch: {:3} | T_Loss {:5} | T_Acc {:5} | V_Loss {:5} | V_Acc {:5} | {:5}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent)

    log_stack.append(log_str)
    
    # 학습 추이 그래프 출력
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpi=99)
    hist_fig.patch.set_facecolor('white')

    # Loss Line 구성
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train_Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid_Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')

    # Acc, Line 구성
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train_Acc', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid_Acc', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')

    # 그래프 출력
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line
    loss_axis.legend(hist_lines, [l.get_label() for l in hist_lines])
    loss_axis.grid()
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()

    # 텍스트 로그 출력
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))):
        print(log_stack[idx])

In [25]:
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

# 학습 알고리즘
def epoch(data_loader, mode = 'train'):
    global epoch_cnt

    # 사용되는 변수 초기화
    iter_loss, iter_acc, last_grad_performed = [], [], False

    # 1 iteration 학습 알고리즘(for문을 나오면 1 epoch 완료)
    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)

        # 1. Feed-forward
        if mode == 'train':
            net.train()
        else:
            # 학습때만 쓰이는 Dropout, Batch Mormalization을 미사용
            net.eval()

        result = net(data) # 1 Batch에 대한 결과가 모든 Class에 대한 확률값으로
        _, out = torch.max(result, 1) # result에서 최대 확률값을 기준으로 예측 class 도출( _ : 값 부분은 필요 없음, out : index 중 가장 큰 하나의 데이터)

        # 2. Loss 계산
        loss = loss_fn(result, label) # GT 와 Label 비교하여 Loss 산정
        iter_loss.append(loss.item()) # 학습 추이를 위하여 Loss를 기록

        # 3. 역전파 학습 후 Gradient Descent
        if mode == 'train':
            optim.zero_grad() # 미분을 통해 얻은 기울기를 초기화 for 다음 epoch
            loss.backward() # 역전파 학습
            optim.step() # Gradient Descent 수행
            last_grad_performed = True # for문을 나가면 epoch 카운터 += 1

        # 4. 정확도 계산
        acc_partial = (out == label).float().sum() # GT == Label 인 개수
        acc_partial = acc_partial / len(label) # ( TP / (TP + TM)) 해서 정확도 산출
        iter_acc.append(acc_partial.item()) # 학습 추이를 위하여 Acc. 기록

    # 역전파 학습 후 Epoch 카운터 += 1
    if last_grad_performed:
        epoch_cnt += 1

    clear_memory()

    # loss와 acc의 평균값 for 학습추이 그래프, 모든 GT와 Label 값 for 컨퓨전 매트릭스
    return np.average(iter_loss), np.average(iter_acc)

def epoch_not_finished():
    # 에폭이 끝남을 알림
    return epoch_cnt < maximum_epoch

In [26]:
# Training initialization
init_model()
init_epoch()
init_log()
maximum_epoch = EPOCH

# Training iteration

while epoch_not_finished():
    start_time = time.time()

    tloss, tacc = epoch(train_loader, mode = 'train')

    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)

    with torch.no_grad():
        vloss, vacc = epoch(val_loader, mode = 'val')
        record_valid_log(vloss, vacc)

    print_log()

print('\n Training completed!')

RuntimeError: stack expects each tensor to be equal size, but got [10, 58] at entry 0 and [7, 58] at entry 2

In [ ]:
# 정확도 검증
with torch.no_grad():
    test_loss, test_acc = epoch(test_loader, mode = 'test')
    test_acc = round(test_acc, 4)
    test_loss = round(test_loss, 4)
    print('Test Acc.: {}'.format(test_acc))
    print('Test Loss: {}'.format(test_loss))

In [ ]:
# 영상 resize 및 추출
test_video_name = 'C_3_12_43_BU_SMC_10-14_12-17-14_CC_RGB_DF2_F2'
test_video_path = f'/content/drive/MyDrive/Colab_Notebooks/Anomaly Detection/{test_video_name}.mp4'
cv2.destroyAllWindows()
cap = cv2.VideoCapture(test_video_path)
img_list = []

if cap.isOpened():

    while True:
        ret, img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            img_list.append(img)
            # cv2_imshow(img)
            # cv2.waitKey(1)
        else:
            break

cap.release()
cv2.destroyAllWindows()

print('저장된 frame의 개수: {}'.format(len(img_list)))